In [0]:
from pyspark.sql.functions import *

In [0]:
df_customers = spark.read.format("delta").load("abfss://olist-data@retailds.dfs.core.windows.net/bronze/customers")

In [0]:
df_customers.printSchema()

In [0]:
df_customers.display()

In [0]:
df_customers.count()

In [0]:
df_customers.select(col("customer_id")).distinct().count()

In [0]:
df_customers.select([
    count(when(col(c).isNull(), c)).alias(c)
    for c in df_customers.columns
]).display()

In [0]:
df_customers = df_customers.filter(col("customer_id").isNotNull())

In [0]:
df_customers.groupBy("customer_id").count().filter(col("count") > 1).display()

In [0]:
df_customers = df_customers.dropDuplicates(subset=["customer_id"])

In [0]:
df_customers.groupBy("customer_unique_id").count().filter(col("count") > 1).display()

In [0]:
df_customers.groupBy("customer_id","customer_unique_id").count().filter(col("count")>1).display()

In [0]:
df_customers = df_customers.withColumn("customer_city", lower(trim(col("customer_city"))))\
    .withColumn("customer_state", upper(trim(col("customer_state"))))

In [0]:
df_customers.write\
    .format("delta")\
    .mode("overwrite")\
    .option("overwriteSchema", "true")\
    .save("abfss://olist-data@retailds.dfs.core.windows.net/silver/customers")